**Inicialmente vamos a importar las librerías necesarias para que el código funcione, en el caso de necesitar librerías nuevas, serán agregadas a medida que sean requeridas**

In [ ]:
!pip install kaggle
# 1. Importar las librerías necesarias
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import json
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer


In [ ]:
os.environ['KAGGLE_CONFIG_DIR']='/content/'

In [ ]:
# Cargar credenciales de Kaggle
data = {"username":"juanmanuellopezarias","key":"3a9f59f3e8a3191d8732a210262ac91a"}
with open('kaggle.json','w') as file:
  json.dump(data, file, indent=4)

**Descargamos los datos de Kaggle**

In [ ]:
!kaggle competitions download -c udea-ai4eng-20242

100% 20.1M/20.1M [00:00<00:00, 106MB/s] 
100% 20.1M/20.1M [00:00<00:00, 103MB/s]


In [ ]:
!unzip \*.zip && rm *.zip

Archive:  udea-ai4eng-20242.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


# **Fase de carga de datos e inspección inicial**

In [ ]:
#Cargar el archivo CSV
data = pd.read_csv('train.csv')
df=data.copy()
df=df.drop(['ID'], axis=1)
df=df.drop(['PERIODO'], axis=1)

#Revisión inicial de los datos
print(df.head())
print(df.info())
print(df.describe())

#Manejo de datos faltantes
#Identificar columnas con valores nulos
print(df.isnull().sum())


          ESTU_PRGM_ACADEMICO ESTU_PRGM_DEPARTAMENTO  \
0                  ENFERMERIA                 BOGOTÁ   
1                     DERECHO              ATLANTICO   
2       MERCADEO Y PUBLICIDAD                 BOGOTÁ   
3  ADMINISTRACION DE EMPRESAS              SANTANDER   
4                  PSICOLOGIA              ANTIOQUIA   

             ESTU_VALORMATRICULAUNIVERSIDAD ESTU_HORASSEMANATRABAJA  \
0  Entre 5.5 millones y menos de 7 millones       Menos de 10 horas   
1  Entre 2.5 millones y menos de 4 millones                       0   
2  Entre 2.5 millones y menos de 4 millones         Más de 30 horas   
3  Entre 4 millones y menos de 5.5 millones                       0   
4  Entre 2.5 millones y menos de 4 millones     Entre 21 y 30 horas   

  FAMI_ESTRATOVIVIENDA FAMI_TIENEINTERNET                 FAMI_EDUCACIONPADRE  \
0            Estrato 3                 Si    Técnica o tecnológica incompleta   
1            Estrato 3                 No      Técnica o tecnológica compl

# **Fase de imputación de datos y tratamiento especial de algunas variables categóricas binarias**

In [ ]:
#Variables Categoricas
ccols = [i for i in df.columns if not i in df._get_numeric_data()]
print (ccols)
#Valor mas repetido
valor_comun = df[ccols].mode().iloc[0]
valor_comun
#Reemplazar datos faltantes con los mas repetidos
for column in ccols:
  df[column].fillna(valor_comun[column], inplace=True)

df

['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE', 'ESTU_PAGOMATRICULAPROPIO', 'RENDIMIENTO_GLOBAL']


<ipython-input-7-f107b74e0d01>:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(valor_comun[column], inplace=True)


,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
0,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Postgrado,No,medio-alto
1,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Técnica o tecnológica incompleta,No,bajo
2,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,bajo
3,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Secundaria (Bachillerato) completa,No,alto
4,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Primaria completa,No,medio-bajo
...,...,...,...,...,...,...,...,...,...,...
692495,BIOLOGIA,LA GUAJIRA,Entre 500 mil y menos de 1 millón,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) incompleta,Si,medio-alto
692496,PSICOLOGIA,NORTE SANTANDER,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Primaria incompleta,Secundaria (Bachillerato) incompleta,No,bajo
692497,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Menos de 10 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) incompleta,Si,medio-bajo
692498,PSICOLOGIA,TOLIMA,Entre 2.5 millones y menos de 4 millones,Menos de 10 horas,Estrato 1,No,Primaria completa,Primaria completa,Si,bajo


 Tratamiento especial de la columna **ESTU_PRGM_ACADEMICO** para reducir las columnas al hacer el One Hot encoding

In [ ]:
def categorizar_programa(academico):
    academico = academico.upper()  # Convertir a mayúsculas para evitar distinciones de minúsculas/mayúsculas

    if academico.startswith('INGE'):
        return 'INGENIERIA'
    elif academico.startswith('LIC'):
        return 'LICENCIATURA'
    elif academico.startswith('ADM'):
        return 'ADMINISTRACION'
    elif academico.startswith('CONT'):
        return 'CONTADURIA'
    elif academico.startswith('COMU'):
        return 'COMUNICACION'
    elif 'DEPORT' in academico:
        return 'DEPORTE'
    elif 'PSIC' in academico:
        return 'PSICOLOGIA'
    elif 'TRABAJO SOCIAL' in academico:
        return 'TRABAJO SOCIAL'
    elif 'DISE' in academico:
        return 'DISEÑO'
    elif 'TERAP' in academico:
        return 'TERAPIA'
    elif 'GASTRON' in academico:
        return 'GASTRONOMIA'
    elif 'ZOOTECNIA' in academico:
        return 'VETERINARIA Y ZOOTECNIA'
    elif 'NUTRIC' in academico:
        return 'NUTRICION'
    elif 'MARKETING' in academico or 'PUBLICIDAD' in academico or 'MERCAD' in academico:
        return 'MARKETING'
    elif 'MATE' in academico or 'ESTA' in academico:
        return 'MATEMATICAS Y ESTADISTICA'
    elif 'ARTE' in academico:
        return 'ARTES'
    elif 'PERIOD' in academico:
        return 'PERIODISMO'
    elif 'LITER' in academico:
        return 'LITERATURA'
    elif 'MUS' in academico or 'MÚS' in academico:
        return 'MUSICA'
    elif 'ECONO' in academico or 'NEG' in academico or 'COME' in academico or 'FIN' in academico:
        return 'ECONOMIA Y NEGOCIOS'
    elif 'GEO' in academico:
        return 'CIENCIAS DE LA TIERRA'
    elif 'CINE' in academico or 'RADIO' in academico or 'TELEV' in academico or 'AUDIOV' in academico or 'FOTOG' in academico:
        return 'CIENCIAS AUDIOVISUALES'
    elif 'DERECHO' in academico:
        return 'DERECHO'
    elif 'POLIT' in academico or 'POLÍT' in academico:
        return 'CIENCIAS POLITICAS'
    elif 'QUIM' in academico or 'QUÍM' in academico:
        return 'QUIMICA'
    elif 'CONST' in academico:
        return 'CONSTRUCCION'
    elif 'TURIS' in academico:
        return 'TURISMO'
    elif 'BIBLIO' in academico:
        return 'BIBLIOTECOLOGIA'
    elif 'FILOS' in academico:
        return 'FILOSOFIA'
    elif 'FILOL' in academico:
        return 'FILOLOGIA'
    elif 'ODONTOLOG' in academico:
        return 'ODONTOLOGIA'
    elif 'BIOL' in academico:
        return 'BIOLOGIA Y MICROBIOLOGIA'
    elif academico.startswith('ECOL'):
        return 'ECOLOGIA'
    elif 'BACT' in academico:
        return 'BACTERIOLOGIA'
    elif 'CULT' in academico:
        return 'CIENCIAS CULTURALES'
    elif 'INSTRUMENTACION' in academico:
        return 'INSTRUMENTACION QUIRURGICA'
    elif 'MILIT' in academico or 'NAVALES' in academico or 'NAU' in academico:
        return 'CIENCIAS MARITIMAS Y AERONAUTICAS'
    else:
        return academico  # Si no coincide con ninguna de las condiciones, se deja igual
#categorizar academico
df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(categorizar_programa)
# Usamos una lista de patrones que deseamos reemplazar y los eliminamos en una sola llamada
patterns = ['PROFESIONAL EN ', 'PROFESIONAL DE ', 'PROFESIONAL ', 'PROGRAMA ']
# Reemplazamos todos los patrones con un espacio vacío usando regex
df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].replace(patterns, '', regex=True)
# Aplicar la función a la columna
df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(categorizar_programa)

 Tratamiento de las columnas con valores **binarios** *(si no)*, se decide en la columna de **ESTU_HORASSEMANATRABAJA**, convertir a binaria (Trabajo o no )


In [ ]:
df["FAMI_TIENEINTERNET"] = df["FAMI_TIENEINTERNET"].replace({'Si': 1, 'No': 0})
df["ESTU_PAGOMATRICULAPROPIO"] = df["ESTU_PAGOMATRICULAPROPIO"].replace({'Si': 1, 'No': 0})
df['ESTU_HORASSEMANATRABAJA'] = df['ESTU_HORASSEMANATRABAJA'].apply(lambda x: 1 if x != 0 else 0)

<ipython-input-9-0ffac0788bdd>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["FAMI_TIENEINTERNET"] = df["FAMI_TIENEINTERNET"].replace({'Si': 1, 'No': 0})
<ipython-input-9-0ffac0788bdd>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["ESTU_PAGOMATRICULAPROPIO"] = df["ESTU_PAGOMATRICULAPROPIO"].replace({'Si': 1, 'No': 0})


Identificamos las columnas categóricas eliminamos las que tienen un **tratamiento especial** ya realizado

In [ ]:
# Identificar columnas categóricas
categorical_columns = df.select_dtypes(include=['object']).columns


# **One-Hot Encoding y Normalización**

Aplicamos **One-Hot Encoding** a las columnas categóricas restantes, normalizamos características numéricas (si aun existen)

In [ ]:
# Aplicar OneHotEncoder a las columnas categóricas
encoder = OneHotEncoder(sparse_output=False, drop='first')  # Eliminar una categoría para evitar multicolinealidad
df_encoded = pd.DataFrame(encoder.fit_transform(df[categorical_columns]),
                          columns=encoder.get_feature_names_out(categorical_columns))

#Agregamos las columnas con tratamiento especial anteriormente realizado
df_encoded = pd.concat([df['FAMI_TIENEINTERNET'], df['ESTU_PAGOMATRICULAPROPIO'], df_encoded], axis=1)
df_encoded = pd.concat([df['ESTU_HORASSEMANATRABAJA'], df_encoded], axis=1)



Unificamos los Dataframes creados y codificados, añadiendo la columna **ID y PERIODO** que borramos al inicio para evitar una normalización de estas características numéricas.

# **Unificación y visualización Final de los Datos**

In [ ]:
#Concatenar las columnas numéricas escaladas y las categóricas codificadas
df_preprocessed = df_encoded
df_preprocessed = pd.concat([data['ID'], df_preprocessed], axis=1)
df_preprocessed = pd.concat([data['PERIODO'], df_preprocessed], axis=1)
# Cambiar la configuración de pandas para mostrar todas las columnas
pd.set_option('display.max_columns', None)


Mostramos el **Dataframe Final**

In [ ]:
df_preprocessed.head()

,PERIODO,ID,ESTU_HORASSEMANATRABAJA,FAMI_TIENEINTERNET,ESTU_PAGOMATRICULAPROPIO,ESTU_PRGM_ACADEMICO_ADMINISTRACION,ESTU_PRGM_ACADEMICO_AGROINDUSTRIA,ESTU_PRGM_ACADEMICO_AGRONOMIA,ESTU_PRGM_ACADEMICO_AGRONOMIA DEL TROPICO HUMEDO,ESTU_PRGM_ACADEMICO_ANIMACIÓN,ESTU_PRGM_ACADEMICO_ANTROPOLOGIA,ESTU_PRGM_ACADEMICO_ANTROPOLOGÍA,ESTU_PRGM_ACADEMICO_ARCHIVISTICA,ESTU_PRGM_ACADEMICO_ARQUEOLOGIA,ESTU_PRGM_ACADEMICO_ARQUITECTURA,ESTU_PRGM_ACADEMICO_ARTES,ESTU_PRGM_ACADEMICO_ASTRONOMIA,ESTU_PRGM_ACADEMICO_BACTERIOLOGIA,ESTU_PRGM_ACADEMICO_BIBLIOTECOLOGIA,ESTU_PRGM_ACADEMICO_BIOINGENIERIA,ESTU_PRGM_ACADEMICO_BIOINGENIERÍA,ESTU_PRGM_ACADEMICO_BIOLOGIA Y MICROBIOLOGIA,ESTU_PRGM_ACADEMICO_BIOTECNOLOGIA,ESTU_PRGM_ACADEMICO_CIENCIAS AMBIENTALES,ESTU_PRGM_ACADEMICO_CIENCIAS AUDIOVISUALES,ESTU_PRGM_ACADEMICO_CIENCIAS BIBLICAS,ESTU_PRGM_ACADEMICO_CIENCIAS CULTURALES,ESTU_PRGM_ACADEMICO_CIENCIAS DE LA COMPUTACION,ESTU_PRGM_ACADEMICO_CIENCIAS DE LA COMPUTACIÓN E INTELIGENCIA ARTIFICIAL,ESTU_PRGM_ACADEMICO_CIENCIAS DE LA INFORMACION Y LA DOCUMENTACION,ESTU_PRGM_ACADEMICO_CIENCIAS DE LA TIERRA,ESTU_PRGM_ACADEMICO_CIENCIAS MARITIMAS Y AERONAUTICAS,ESTU_PRGM_ACADEMICO_CIENCIAS POLITICAS,ESTU_PRGM_ACADEMICO_COMPUTACIÓN CIENTÍFICA,ESTU_PRGM_ACADEMICO_COMUNICACION,ESTU_PRGM_ACADEMICO_CONSTRUCCION,ESTU_PRGM_ACADEMICO_CONTADURIA,ESTU_PRGM_ACADEMICO_CRIMINALISTICA,ESTU_PRGM_ACADEMICO_DANZA,ESTU_PRGM_ACADEMICO_DANZA Y DIRECCIÓN COREOGRÁFICA,ESTU_PRGM_ACADEMICO_DEPORTE,ESTU_PRGM_ACADEMICO_DERECHO,ESTU_PRGM_ACADEMICO_DESARROLLO FAMILIAR,ESTU_PRGM_ACADEMICO_DESARROLLO TERRITORIAL,ESTU_PRGM_ACADEMICO_DIRECCION DE BANDA,ESTU_PRGM_ACADEMICO_DISEÑO,ESTU_PRGM_ACADEMICO_ECOLOGIA,ESTU_PRGM_ACADEMICO_ECONOMIA Y NEGOCIOS,ESTU_PRGM_ACADEMICO_ENFERMERIA,ESTU_PRGM_ACADEMICO_ENFERMERÍA,ESTU_PRGM_ACADEMICO_FARMACIA,ESTU_PRGM_ACADEMICO_FILOLOGIA,ESTU_PRGM_ACADEMICO_FILOSOFIA,ESTU_PRGM_ACADEMICO_FISICA,ESTU_PRGM_ACADEMICO_FONOAUDIOLOGIA,ESTU_PRGM_ACADEMICO_FÍSICA,ESTU_PRGM_ACADEMICO_GASTRONOMIA,ESTU_PRGM_ACADEMICO_GERENCIA EN SISTEMAS DE INFORMACION EN SALUD,ESTU_PRGM_ACADEMICO_GERONTOLOGIA,ESTU_PRGM_ACADEMICO_GESTION EMPRESARIAL,ESTU_PRGM_ACADEMICO_GESTION Y DESARROLLO URBANOS,ESTU_PRGM_ACADEMICO_GESTIÓN DE SISTEMAS DE INFORMACIÓN,ESTU_PRGM_ACADEMICO_GESTIÓN DE LA SEGURIDAD Y LA SALUD LABORAL,ESTU_PRGM_ACADEMICO_GOBIERNO Y ASUNTOS PUBLICOS,ESTU_PRGM_ACADEMICO_GOBIERNO Y RELACIONES INTERNACIONALES,ESTU_PRGM_ACADEMICO_HISTORIA,ESTU_PRGM_ACADEMICO_HISTORIA Y ARCHIVISTICA,ESTU_PRGM_ACADEMICO_HISTORIA Y PATRIMONIO,ESTU_PRGM_ACADEMICO_INGENIERIA,ESTU_PRGM_ACADEMICO_INSTRUMENTACION QUIRURGICA,ESTU_PRGM_ACADEMICO_INTRUMENTACION QUIRURGICA,ESTU_PRGM_ACADEMICO_INVESTIGACION CRIMINAL,ESTU_PRGM_ACADEMICO_INVESTIGACIÓN CRIMINAL,ESTU_PRGM_ACADEMICO_JURISPRUDENCIA,ESTU_PRGM_ACADEMICO_LENGUAS MODERNAS,ESTU_PRGM_ACADEMICO_LICENCIATURA,ESTU_PRGM_ACADEMICO_LINGUISTICA,ESTU_PRGM_ACADEMICO_LITERATURA,ESTU_PRGM_ACADEMICO_LOGISTICA EMPRESARIAL,ESTU_PRGM_ACADEMICO_MARKETING,ESTU_PRGM_ACADEMICO_MATEMATICAS Y ESTADISTICA,ESTU_PRGM_ACADEMICO_MEDICINA,ESTU_PRGM_ACADEMICO_MEDICINA VETERINARIA,ESTU_PRGM_ACADEMICO_MUSICA,ESTU_PRGM_ACADEMICO_NARRATIVAS DIGITALES,ESTU_PRGM_ACADEMICO_NUTRICION,ESTU_PRGM_ACADEMICO_OCEANOGRAFIA FISICA,ESTU_PRGM_ACADEMICO_OCEANOGRAFÍA,ESTU_PRGM_ACADEMICO_ODONTOLOGIA,ESTU_PRGM_ACADEMICO_OPTOMETRIA,ESTU_PRGM_ACADEMICO_OPTOMETRÍA,ESTU_PRGM_ACADEMICO_PEDAGOGÍA,ESTU_PRGM_ACADEMICO_PERIODISMO,ESTU_PRGM_ACADEMICO_PLANEACION Y DESARROLLO SOCIAL,ESTU_PRGM_ACADEMICO_PRODUCCION AGROINDUSTRIAL,ESTU_PRGM_ACADEMICO_PSICOLOGIA,ESTU_PRGM_ACADEMICO_QUIMICA,ESTU_PRGM_ACADEMICO_QU¿MICA FARMAC¿UTICA,ESTU_PRGM_ACADEMICO_REALIZACIÓN EN ANIMACIÓN,ESTU_PRGM_ACADEMICO_RECREACION,ESTU_PRGM_ACADEMICO_RELACIONES INTERNACIONALES,ESTU_PRGM_ACADEMICO_SALUD OCUPACIONAL,ESTU_PRGM_ACADEMICO_SEGURIDAD Y SALUD EN EL TRABAJO,ESTU_PRGM_ACADEMICO_SISTEMAS DE INFORMACION Y DOCUMENTACION,ESTU_PRGM_ACADEMICO_SOCIOLOGIA,ESTU_PRGM_ACADEMICO_SOCIOLOGÍA,ESTU_PRGM_ACADEMICO_TEOLOGIA,ESTU_PRGM_ACADEMICO_TEOLOGÍA,ESTU_PRGM_ACADE